In [1]:
from pathlib import Path

import timm
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from tooth_crop_dataset import ToothCropClassDataset
from utils.vit import train, test

log_dir = Path('runs') / 'swin_base_patch4_window7_224_in22k'
writer = SummaryWriter(log_dir=log_dir)


/opt/miniconda3/envs/pt113/lib/python3.9/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
# Data
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Preprocess
transform = transforms.Compose([
    transforms.ToTensor(),
    # (lambda image: padding_to_size(image, 224)),
    transforms.Resize(size=(224, 224)),
    # transforms.Resize(size=(256, 256)),
    # transforms.Resize(size=(384, 384)),
    transforms.Normalize(mean=0.5, std=0.5),
])
target_transform = transforms.Compose([
    (lambda y: torch.Tensor(y)),
])

# Hyperparameter
epoch_num = 300
batch_size = 16
num_workers = 0
train_test_split = 0.8


In [3]:
dataset = ToothCropClassDataset(root='../preprocess/runs/detect/predict4/crops', transform=transform,
                                target_transform=target_transform, yolov8=True)

dataset_size = len(dataset)
train_size = int(train_test_split * dataset_size)
test_size = dataset_size - train_size

train_set, test_set = random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size,
                                           shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size,
                                          shuffle=True, num_workers=num_workers)

classes = dataset.mlb.classes_

train_label_count = torch.zeros(len(classes))
for x, y in train_loader:
    train_label_count += y.sum(axis=0)

test_label_count = torch.zeros(len(classes))
for x, y in test_loader:
    test_label_count += y.sum(axis=0)

print(classes)
print(train_label_count)
print(test_label_count)


Total data in 2453
['R.R' 'caries' 'crown' 'endo' 'filling' 'post']
tensor([  81.,  139.,  459.,  552., 1130.,  322.])
tensor([ 19.,  36., 105., 144., 288.,  86.])


In [4]:
# model = timm.create_model('mobilenetv2_100', num_classes=6, pretrained=True)
# model = timm.create_model('swin_base_patch4_window7_224', num_classes=6, pretrained=True)
# model = timm.create_model('swin_s3_base_224', num_classes=6, pretrained=True)
model = timm.create_model('swin_base_patch4_window7_224_in22k', num_classes=6, pretrained=False)
# model.load_state_dict(torch.load('./runs/swin_base_patch4_window7_224_in22k/trained-net.pt'))
model.to(device)

pos_weight = torch.tensor([1, 1, 1, 1, 1, 1]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
SGD_optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


/opt/miniconda3/envs/pt113/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
for t in tqdm(range(epoch_num)):
    # for t in tqdm(range(epoch_num, epoch_num + 10)):
    # print(f"Epoch {t + 1}\n-------------------------------")
    train(train_loader, model, criterion, opt, writer=writer, epoch=t, device=device)
    test(test_loader, model, criterion, len(classes), device=device, writer=writer, epoch=t, classes=classes,
         threshold=0.95)

writer.close()
print("Done!")

print('Finished Training')
# save your improved network
torch.save(model.state_dict(), log_dir / 'trained-net.pt')


  0%|          | 0/300 [09:18<?, ?it/s]


KeyboardInterrupt: 